<a href="https://colab.research.google.com/github/vineethpetnakota/emotion-detection-using-social-media-data/blob/main/llm_huggingface_aug26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
!pip install langchain_community

In [108]:
from langchain_community.document_loaders.csv_loader import CSVLoader


file_path = (
    "/content/instadata100_filtered.csv"
)

loader = CSVLoader(file_path=file_path)
data = loader.load()

In [109]:
!pip install langchain_huggingface

In [110]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings()

In [111]:
!pip install langchain-chroma

In [113]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=data, embedding=embeddings_model)

In [114]:
from google.colab import userdata
key = userdata.get('mykey')

In [115]:
!pip install langchain-groq

In [116]:
import getpass
import os

os.environ["GROQ_API_KEY"] = key

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [117]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Consider yourself as a psychiatrist for answering the emotional state of a particular user mentioned in the prompt. "
    "Use the fiedl owner_username field as user names"
    "Also give guidance to make the emotional state better"
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

#retriever = vectorstore.as_retriever()
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response1 = rag_chain.invoke({"input": "List all usernames from the dataset, using the 'owner_username' field. Ensure no users are missed."})
print(response1["answer"])

# Prompt to retrieve the emotional state of a specific user
user = input("Enter the name of the user: ").strip().lower()  # Ensure consistency in input
response = rag_chain.invoke({"input": f"What is the emotional state of the user {user}?"})
print(response["answer"])


Here is the list of usernames from the dataset:

1. digernes
2. christinehmcconnell
3. erin.adlina

Let me know if you need any further assistance!
Enter the name of the user: christinehmcconnell
I apologize, but there is no user named "christinehmcconnell" in the provided data. The usernames mentioned are "sheerworldwide" and "crazyjewishmom". If you meant to ask about one of these users, I'd be happy to help.
